In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.utils.data as Data
import torch.nn as nn
import numpy as np
from scipy.stats import norm
from scipy.stats import chi2
import random
from scipy.linalg import sqrtm
from scipy.linalg import toeplitz
from scipy.spatial.distance import pdist, squareform
import math
from sklearn.model_selection import KFold
from torchvision import models



#network
class Net(nn.Module):
    def __init__(self,n_features,num=25, n_out=1,rate = 0):
        super(Net,self).__init__()
        self.out = nn.Sequential(

            nn.Dropout(rate),
            nn.Linear(n_features,num),
            nn.ReLU(),

            nn.Dropout(rate),
            nn.Linear(num, num),
            nn.ReLU(),

            nn.Dropout(rate),
            nn.Linear(num, num),
            nn.ReLU(),

            nn.Dropout(rate),
            nn.Linear(num, num),
            nn.ReLU(),

            nn.Linear(num, n_out)
        )
    def forward(self,x):
        x = self.out(x)
        return x

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


#early stopping
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=20, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 20
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            #self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

#Example A1
def Creat_data(n,p1,p2, case, a, rho=0.3):
    p = p1+p2
    p0 = p//2
    temp = rho**np.linspace(0,p-1,p)
    Sigma =  toeplitz(temp,temp)
    Sigmasqrt =  torch.from_numpy(sqrtm(Sigma)).type(torch.FloatTensor)
    x = torch.randn(n,p)
    x = torch.mm(x,Sigmasqrt)
    if case == 1:#sparse
        beta = torch.cat([torch.ones(2,1),torch.zeros(p1-2,1),torch.ones(2,1)*a/math.sqrt(2),torch.zeros(p2-2,1)],dim=0)
        epsilon = torch.randn(n,1)*0.5 #homo
        #epsilon = torch.randn(n,1)*torch.unsqueeze(torch.exp(x[:,2])/(1+torch.exp(x[:,2])),dim=1) #hete
        y = torch.mm(x,beta)+epsilon

    elif case == 2:#dense
        beta = torch.cat([torch.ones(2,1),torch.zeros(p1-2,1),torch.ones(p2,1)*a/math.sqrt(p2)],dim=0)
        epsilon = torch.randn(n,1)*0.5
        #epsilon = torch.randn(n,1)*torch.unsqueeze(torch.exp(x[:,2])/(1+torch.exp(x[:,2])),dim=1) #hete
        y = torch.mm(x,beta)+epsilon

    x = (x - torch.mean(x,dim=0))/torch.std(x,dim=0)
    y = (y-torch.mean(y))/torch.std(y)
    z = x[:,:p1]
    w = x[:,p1:]
    return z,w,y

#Example A2
def Creat_data2(n,p1,p2, case, a, rho=0.3):
    p = p1+p2
    p0 = p//2
    temp = rho**np.linspace(0,p-1,p)
    Sigma =  toeplitz(temp,temp)
    Sigmasqrt =  torch.from_numpy(sqrtm(Sigma)).type(torch.FloatTensor)
    x = torch.randn(n,p)
    x = torch.mm(x,Sigmasqrt)
    z = x[:,:p1]
    w = x[:,p1:]
    if case == 1:#sparse
        beta = torch.cat([torch.ones(5,1)*a/math.sqrt(5),torch.zeros(p2-5,1)],dim=0)#####
        epsilon = torch.randn(n,1)*0.5
        y = torch.unsqueeze(z[:,0]+z[:,1],dim=1) +torch.mm(w,beta)**2+epsilon

    elif case == 2:#dense
        beta = torch.cat([torch.ones(int(np.floor(p2/2)),1)*a/math.sqrt(int(np.floor(p2/2))), torch.zeros(p2-int(np.floor(p2/2)),1)],dim=0)
        epsilon = torch.randn(n,1)*0.5
        y = torch.unsqueeze(z[:,0]+z[:,1],dim=1) +torch.mm(w,beta)**2+epsilon

    z = x[:,:p1]
    w = x[:,p1:]
    return z,w,y

#Example A3
def Creat_data3(n,p1,p2, case, a, rho=0.3):
    p = p1+p2
    p0 = p//2
    temp = rho**np.linspace(0,p-1,p)
    Sigma =  toeplitz(temp,temp)
    Sigmasqrt =  torch.from_numpy(sqrtm(Sigma)).type(torch.FloatTensor)
    x = torch.randn(n,p)
    x = torch.mm(x,Sigmasqrt)
    z = x[:,:p1]
    w = x[:,p1:]
    if case == 1:#sparse
        beta = torch.cat([torch.ones(2,1)*a,torch.zeros(p2-2,1)],dim=0)
        epsilon = torch.randn(n,1)*0.5
        y = torch.unsqueeze(z[:,0]-z[:,1],dim=1) + torch.mm(torch.exp(w),beta)+epsilon

    elif case == 2:#dense
        beta = torch.cat([torch.ones(p2,1)*a],dim=0)
        epsilon = torch.randn(n,1)*0.5
        y = torch.unsqueeze(z[:,0]-z[:,1],dim=1) + torch.mm(torch.exp(w),beta)+epsilon
    z = x[:,:p1]
    w = x[:,p1:]
    return z,w,y

#single split
def test1(z,w,y,c):
    n = z.shape[0]
    p1 = z.shape[1]
    p2 = w.shape[1]
    x = torch.cat([z,w],1)
    p = p1+p2
    n1 = math.ceil(c*n)

    EPOCH = 200
    BATCH_SIZE =50
    LR = 0.003#0.001
    num = 50

    index = np.random.choice(a=n, size=n1, replace=False)
    x_train = x[index,:]
    x_test = np.delete(x,index,axis=0)
    z_train = z[index,:]
    z_test = np.delete(z,index,axis=0)
    y_train = y[index,:]
    y_test = np.delete(y,index,axis=0)

    net1= Net(p1,num=num,rate = 0)
    loss_func=nn.MSELoss()
    optimizer=torch.optim.Adam(net1.parameters(),lr=LR,weight_decay = 1e-8)
    early_stopping = EarlyStopping(patience=EPOCH, verbose=False)
    train_result = np.zeros(EPOCH)
    test_result = np.zeros(EPOCH)
    train_data = Data.TensorDataset(z_train , y_train)
    train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
    #----------------------------------------------------------------
    for epoch in range(EPOCH):
        #---------------------------------------------------
        net1.train()
        for step, (b_x, b_y) in enumerate(train_loader):
            predicton=net1(b_x)
            loss=loss_func(predicton,b_y)
            optimizer.zero_grad()           # clear gradients for this training step
            loss.backward()                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients

        net1.eval()
        train_result[epoch] = loss_func(net1(z_train),y_train )
        valid_loss = loss_func(net1(z_test),y_test )
        test_result[epoch] = valid_loss
        #if (epoch+1)%10==0:
            #print('Epoch: ', epoch+1, '| train loss: %.4f' % loss.data.numpy() )
        early_stopping(valid_loss, net1)

        if early_stopping.early_stop:
            break

    net1.load_state_dict(torch.load('checkpoint.pt'))

    net1.eval()
    h_train = net1(z_train)
    y_tilde_train = y_train - h_train
    h_test = net1(z_test)
    y_tilde_test = y_test - h_test

    net2 = Net(p,num=num,rate=0)
    BATCH_SIZE = n1
    loss_func=nn.MSELoss()
    optimizer=torch.optim.Adam(net2.parameters(),lr=LR,weight_decay = 1e-4)
    early_stopping = EarlyStopping(patience=7, verbose=False)
    train_result = np.zeros(EPOCH)
    test_result = np.zeros(EPOCH)
    train_data2 = Data.TensorDataset(x_train, y_tilde_train)
    train_loader2 = Data.DataLoader(dataset=train_data2, batch_size=BATCH_SIZE, shuffle=True)
    #----------------------------------------------------------------
    for epoch in range(EPOCH):
        #---------------------------------------------------
        net2.train()
        for step, (b_x, b_y) in enumerate(train_loader2):
            predicton=net2(b_x)
            loss=loss_func(predicton,b_y)
            optimizer.zero_grad()           # clear gradients for this training step
            loss.backward(retain_graph=True)                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients

        net2.eval()
        train_result[epoch] = loss_func(net2(x_train),y_tilde_train)
        valid_loss = loss_func(net2(x_test),y_tilde_test)
        test_result[epoch] = valid_loss
        early_stopping(valid_loss, net2)

        if early_stopping.early_stop:
            #print("Early stopping")
            break

    net2.load_state_dict(torch.load('checkpoint.pt'))

    net2.eval()
    f_hat = net2(x_test).detach().numpy()
    Tn = np.sum((f_hat-np.mean(y_tilde_test.detach().numpy()))**2)/np.var( y_tilde_test.detach().numpy() )
    Tn_enhance = Tn+np.sum(f_hat**2)
    return([Tn,Tn_enhance])

####select the splitting ratio
def choose_c(z,w,y):
    c_list = [3/4,4/5,5/6,6/7,7/8,8/9,9/10,19/20]
    c_num = len(c_list)
    n = z.shape[0]
    p1 = z.shape[1]
    p2 = w.shape[1]
    x = torch.cat([z,w],1)
    p = p1+p2

    MM = 200
    temp = np.zeros(MM)
    ans = np.zeros(c_num)
    j = 0
    for c in range(c_num):
        cc = c_list[c]
        for m in range(MM):
            permu = np.random.choice(a=n, size=n, replace=False)
            ww = w[permu,:]
            temp[m] = test1(z,ww,y,cc)[0]
            #print("\r",cc,'  ---  times:', m+1,end="",flush=True)
        #print('\n',cc,' --- ',np.mean(temp>chi2.ppf(0.95,df=1)))
        ans[j] = np.mean(temp>chi2.ppf(0.95,df=1))
        if ans[j]<=0.05:
            return (cc)
        j = j+1

    if min(ans)>0.05:
        return (c_list[np.argmin(ans)])

#Multiple data splitting test
def Cauchy_test1(z,w,y,c,B=5):
    pv = np.zeros([2,B])
    Tx = np.zeros([2,B])
    for b in range(B):
        test_result = test1(z,w,y,c)
        pv[0,b] = 1- chi2.cdf(test_result[0],df=1)
        pv[1,b] = 1- chi2.cdf(test_result[1],df=1)
        Tx[0,b] = np.tan((1/2-pv[0,b])*np.pi)
        Tx[1,b] = np.tan((1/2-pv[1,b])*np.pi)
    Tmean=np.mean(Tx,1)
    Q = 1/2- np.arctan(Tmean)/np.pi
    return(Q)



#single split
def Dai(z,w,y,c=0.5,r = 0.01):
    n = z.shape[0]
    p1 = z.shape[1]
    p2 = w.shape[1]
    x = torch.cat([z,w],1)
    p = p1+p2
    n1 = math.ceil(c*n)

    EPOCH = 200
    BATCH_SIZE =50
    LR = 0.003
    num = 50

    index = np.random.choice(a=n, size=n1, replace=False)
    x_train = x[index,:]
    x_test = np.delete(x,index,axis=0)

    z = torch.cat([z,torch.zeros(n,p2)],1)
    z_train = z[index,:]
    z_test = np.delete(z,index,axis=0)

    y_train = y[index,:]
    y_test = np.delete(y,index,axis=0)

    net1= Net(p,num=num,rate = 0)
    loss_func=nn.MSELoss()
    optimizer=torch.optim.Adam(net1.parameters(),lr=LR,weight_decay = 1e-4)
    early_stopping = EarlyStopping(patience=20, verbose=False)
    train_result = np.zeros(EPOCH)
    test_result = np.zeros(EPOCH)
    train_data = Data.TensorDataset(z_train , y_train)
    train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
    #----------------------------------------------------------------
    for epoch in range(EPOCH):
        #---------------------------------------------------
        net1.train()
        for step, (b_x, b_y) in enumerate(train_loader):
            predicton=net1(b_x)
            loss=loss_func(predicton,b_y)
            optimizer.zero_grad()           # clear gradients for this training step
            loss.backward()                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients

        net1.eval()
        train_result[epoch] = loss_func(net1(z_train),y_train )
        valid_loss = loss_func(net1(z_test),y_test )
        test_result[epoch] = valid_loss
        early_stopping(valid_loss, net1)
        if early_stopping.early_stop:
            break

    net1.load_state_dict(torch.load('checkpoint.pt'))

    net1.eval()
    h_test = net1(z_test)

    net2 = Net(p,num=num,rate=0)
    loss_func=nn.MSELoss()
    optimizer=torch.optim.Adam(net2.parameters(),lr=LR,weight_decay = 1e-4)
    early_stopping = EarlyStopping(patience=20, verbose=False)
    train_result = np.zeros(EPOCH)
    test_result = np.zeros(EPOCH)
    train_data2 = Data.TensorDataset(x_train, y_train)
    train_loader2 = Data.DataLoader(dataset=train_data2, batch_size=BATCH_SIZE, shuffle=True)
    #----------------------------------------------------------------
    for epoch in range(EPOCH):
        #---------------------------------------------------
        net2.train()

        for step, (b_x, b_y) in enumerate(train_loader2):
            predicton=net2(b_x)
            loss=loss_func(predicton,b_y)
            optimizer.zero_grad()           # clear gradients for this training step
            loss.backward(retain_graph=True)                 # backpropagation, compute gradients
            optimizer.step()                # apply gradients

        net2.eval()
        train_result[epoch] = loss_func(net2(x_train),y_train)
        valid_loss = loss_func(net2(x_test),y_test)
        test_result[epoch] = valid_loss
        #if (epoch+1)%10==0:
            #print('Epoch: ', epoch+1, '| train loss: %.4f' % loss.data.numpy() )
        early_stopping(valid_loss, net2)
        if early_stopping.early_stop:
            #print("Early stopping")
            break

    net2.load_state_dict(torch.load('checkpoint.pt'))

    net2.eval()
    f_hat = net2(x_test).detach().numpy()

    n2 = n - n1
    delta = ((y_test-f_hat)**2 - (y_test-h_test)**2 + torch.randn(n2,1)*r).detach().numpy()
    return(np.sum(delta)/n2**(0.5)/np.std(delta))


def Cauchy_Dai(z,w,y,c,r,B=5):
    pv = np.zeros(B)
    Tx = np.zeros(B)
    for b in range(B):
        pv[b] = norm.cdf(Dai(z,w,y,c,r))
        Tx[b] = np.tan((1/2-pv[b])*np.pi)
    Tmean=np.mean(Tx)
    Q = 1/2- np.arctan(Tmean)/np.pi
    return(Q)

setup_seed(123)
MCMC = 500

n_list = [400, 300, 200, 100]

# n = 100
for n in n_list:
    result = np.zeros([MCMC,3])
    for xxx in range(MCMC):
        z,w,y = Creat_data2(n,25,25,case=1,a=0,rho=0.3)
        result[xxx,0] = Dai(z,w,y,c=0.5,r=0.01)
        z,w,y = Creat_data2(n,25,25,case=1,a=0.5,rho=0.3)
        result[xxx,1] = Dai(z,w,y,c=0.5,r=0.01)
        z,w,y = Creat_data2(n,25,25,case=2,a=0.5,rho=0.3)
        result[xxx,2] = Dai(z,w,y,c=0.5,r=0.01)
        print("\r",'  times:', xxx+1,'  ---  ',np.sum(result[0:(xxx+1),:]<norm.ppf(0.05),axis=0)/(xxx+1),end="",flush=True)
    print('\n',np.sum(result<norm.ppf(0.05),axis=0)/MCMC)


    result2 = np.zeros([MCMC,3])
    for xxx in range(MCMC):
        z,w,y = Creat_data2(n,25,25,case=1,a=0,rho=0.3)
        result2[xxx,0] = Cauchy_Dai(z,w,y,c=0.5,r=0.01)
        z,w,y = Creat_data2(n,25,25,case=1,a=0.5,rho=0.3)
        result2[xxx,1] = Cauchy_Dai(z,w,y,c=0.5,r=0.01)
        z,w,y = Creat_data2(n,25,25,case=2,a=0.5,rho=0.3)
        result2[xxx,2] = Cauchy_Dai(z,w,y,c=0.5,r=0.01)
        print("\r",'  times:', xxx+1,'  ---  ',np.sum(result2[0:(xxx+1),:]<0.05,axis=0)/(xxx+1),end="",flush=True)
    print('\n',np.sum(result2<0.05,axis=0)/MCMC)

    np.savetxt("/content/drive/My Drive/Mean_Indep_baseline/Example_A2_Dai_H0_H1_s1_d1_ans_n_" + str(n) + "_python.csv", result, delimiter=",")
    np.savetxt("/content/drive/My Drive/Mean_Indep_baseline/Example_A2_Cauchy_Dai_H0_H1_s1_d1_ans_n_" + str(n) + "_python.csv", result2, delimiter=",")